# Analyze Results of a Hyperparameter Tuning job

Once you have completed a tuning job, (or even while the job is still running) you can use this notebook to analyze the results to understand how each hyperparameter effects the quality of the model.

---
## Set up the environment
To start the analysis, you must pick the name of the hyperparameter tuning job.

In [1]:
import boto3
import sagemaker
import os

region = boto3.Session().region_name
sage_client = boto3.Session().client('sagemaker')

# Fill in your Tuning job name here
tuning_job_name = 'xgboost-200829-0901'

## Track hyperparameter tuning job progress
After you launch a tuning job, you can see its progress by calling describe_tuning_job API. The output from describe-tuning-job is a JSON object that contains information about the current state of the tuning job. You can call list_training_jobs_for_tuning_job to see a detailed list of the training jobs that the tuning job launched.

In [2]:
# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=tuning_job_name)

status = tuning_job_result['HyperParameterTuningJobStatus']
if status != 'Completed':
    print('Reminder: the tuning job has not been completed.')
    
job_count = tuning_job_result['TrainingJobStatusCounters']['Completed']
print("%d training jobs have completed" % job_count)
    
is_minimize = (tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['Type'] != 'Maximize')
objective_name = tuning_job_result['HyperParameterTuningJobConfig']['HyperParameterTuningJobObjective']['MetricName']

9 training jobs have completed


In [3]:
from pprint import pprint
if tuning_job_result.get('BestTrainingJob',None):
    print("Best model found so far:")
    pprint(tuning_job_result['BestTrainingJob'])
else:
    print("No training jobs have reported results yet.")

Best model found so far:
{'CreationTime': datetime.datetime(2020, 8, 29, 9, 5, 16, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'validation:auc',
                                                 'Value': 0.7740319967269897},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2020, 8, 29, 9, 8, 33, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:ap-southeast-1:342474125894:training-job/xgboost-200829-0901-004-6896b295',
 'TrainingJobName': 'xgboost-200829-0901-004-6896b295',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2020, 8, 29, 9, 7, 35, tzinfo=tzlocal()),
 'TunedHyperParameters': {'alpha': '1.5923748561274906',
                          'eta': '0.12392106816315264',
                          'max_depth': '8',
                          'min_child_weight': '8.150098409770688'}}


## Fetch all results as DataFrame
We can list hyperparameters and objective metrics of all training jobs and pick up the training job with the best objective metric.

In [4]:
import pandas as pd

tuner = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

full_df = tuner.dataframe()

if len(full_df) > 0:
    df = full_df[full_df['FinalObjectiveValue'] > -float('inf')]
    if len(df) > 0:
        df = df.sort_values('FinalObjectiveValue', ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest":min(df['FinalObjectiveValue']),"highest": max(df['FinalObjectiveValue'])})
        pd.set_option('display.max_colwidth', -1)  # Don't truncate TrainingJobName        
    else:
        print("No training jobs have reported valid results yet.")
        
df

Number of training jobs with valid objective: 9
{'lowest': 0.7103430032730103, 'highest': 0.7740319967269897}


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:13: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


,alpha,eta,max_depth,min_child_weight,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
5,1.592375,0.123921,8.0,8.150098,xgboost-200829-0901-004-6896b295,Completed,0.774032,2020-08-29 09:07:35+00:00,2020-08-29 09:08:33+00:00,58.0
4,0.509733,0.026814,7.0,8.418714,xgboost-200829-0901-005-12386256,Completed,0.773113,2020-08-29 09:07:23+00:00,2020-08-29 09:08:19+00:00,56.0
1,1.833342,0.070088,3.0,2.673238,xgboost-200829-0901-008-61414774,Completed,0.772901,2020-08-29 09:10:43+00:00,2020-08-29 09:11:34+00:00,51.0
0,1.813342,0.080088,3.0,2.583238,xgboost-200829-0901-009-60362060,Completed,0.772744,2020-08-29 09:10:55+00:00,2020-08-29 09:11:54+00:00,59.0
7,0.372405,0.063271,8.0,7.384094,xgboost-200829-0901-002-03180d0e,Completed,0.772723,2020-08-29 09:04:10+00:00,2020-08-29 09:05:00+00:00,50.0
8,0.611437,0.833140,2.0,3.986192,xgboost-200829-0901-001-aa4a236a,Completed,0.771340,2020-08-29 09:04:05+00:00,2020-08-29 09:04:51+00:00,46.0
6,0.232372,0.533889,6.0,8.405433,xgboost-200829-0901-003-43f16ea2,Completed,0.752221,2020-08-29 09:04:16+00:00,2020-08-29 09:05:12+00:00,56.0
3,0.710212,0.376301,8.0,1.482869,xgboost-200829-0901-006-f498c539,Completed,0.739208,2020-08-29 09:08:03+00:00,2020-08-29 09:08:48+00:00,45.0
2,1.062629,0.964497,8.0,6.310811,xgboost-200829-0901-007-92195b59,Completed,0.710343,2020-08-29 09:10:56+00:00,2020-08-29 09:11:50+00:00,54.0


## See TuningJob results vs time
Next we will show how the objective metric changes over time, as the tuning job progresses.  For Bayesian strategy, you should expect to see a general trend towards better results, but this progress will not be steady as the algorithm needs to balance _exploration_ of new areas of parameter space against _exploitation_ of known good areas.  This can give you a sense of whether or not the number of training jobs is sufficient for the complexity of your search space.

In [7]:
import bokeh
import bokeh.io
bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool

class HoverHelper():

    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append( (k, "@{%s}" % k) )

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools='pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset'):
        return [self.hovertool(), standard_tools]

hover = HoverHelper(tuner)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type='datetime')
p.circle(source=df, x='TrainingStartTime', y='FinalObjectiveValue')
show(p)

Loading BokehJS ...

## Analyze the correlation between objective metric and individual hyperparameters 
Now you have finished a tuning job, you may want to know the correlation between your objective metric and individual hyperparameters you've selected to tune. Having that insight will help you decide whether it makes sense to adjust search ranges for certain hyperparameters and start another tuning job. For example, if you see a positive trend between objective metric and a numerical hyperparameter, you probably want to set a higher tuning range for that hyperparameter in your next tuning job.

The following cell draws a graph for each hyperparameter to show its correlation with your objective metric.

In [8]:
ranges = tuner.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get('Values'):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0           
        vals = hp_range['Values']
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args['x_range'] = vals

    # Now plot it
    p = figure(plot_width=500, plot_height=500, 
               title="Objective vs %s" % hp_name,
               tools=hover.tools(),
               x_axis_label=hp_name, y_axis_label=objective_name,
               **categorical_args)
    p.circle(source=df, x=hp_name, y='FinalObjectiveValue')
    figures.append(p)
show(bokeh.layouts.Column(*figures))